In [2]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import hvplot.pandas  # noqa
import hvplot.dask  # noqa

# Build Dask Cluster
1. Use gui interface to create a new cluster with ~10 workers
2. Use < > to insert an "import Client" statement. This is critical because it is how your script knows to use yoru cluster. 
3. Execute client cell
4. Execute your Dask Cell. Note that once you have a cluster running you do not need to re-import the client. 
5. When finished, always shut down your cluster. 

In [3]:
from dask.distributed import Client

client = Client("tcp://10.0.128.152:44023")
client

OSError: Timed out trying to connect to 'tcp://10.0.128.152:44023' after 10 s: Timed out trying to connect to 'tcp://10.0.128.152:44023' after 10 s: in <distributed.comm.tcp.TCPConnector object at 0x7f7d0d07f5f8>: OSError: [Errno 113] No route to host

In [ ]:
from pylab import rcParams

In [14]:
ds = xr.open_mfdataset('/home/jovyan/data/botpt/minute_mean_dataD/*_resampled.nc', parallel=True
                       ,combine='by_coords', chunks={'index': 80000})
ds

<xarray.Dataset>
Dimensions:          (index: 1145599)
Coordinates:
  * index            (index) datetime64[ns] 2017-06-13 ... 2019-10-14T23:59:00
Data variables:
    bottom_pressure  (index) float32 dask.array<chunksize=(10800,), meta=np.ndarray>

In [20]:
ds = xr.open_mfdataset('/home/jovyan/data/botpt/minute_mean_dataD/*_resampled.nc', parallel=True
                       ,combine='by_coords').chunk(80000)
ds

<xarray.Dataset>
Dimensions:          (index: 1145599)
Coordinates:
  * index            (index) datetime64[ns] 2017-06-13 ... 2019-10-14T23:59:00
Data variables:
    bottom_pressure  (index) float32 dask.array<chunksize=(80000,), meta=np.ndarray>

In [27]:
df = ds.to_dask_dataframe()
df = df.rename(columns={"index": "time"});
df = df.set_index('time')

In [28]:
df.head()

,bottom_pressure
time,
2017-06-13 00:00:00,2259.683105
2017-06-13 00:01:00,2259.678711
2017-06-13 00:02:00,2259.674072
2017-06-13 00:03:00,2259.660400
2017-06-13 00:04:00,2259.666992


In [24]:
df.describe()

,bottom_pressure
npartitions=1,
,float64
,...


In [29]:
df.hvplot(y='bottom_pressure', datashade =True)

:DynamicMap   []
   :RGB   [time,bottom_pressure]   (R,G,B,A)